# DataFrame

http://spark.apache.org/docs/latest/api/python/pyspark.sql.html#pyspark-sql-module

In [70]:
from pyspark.sql import SparkSession
from pyspark.sql import Row

spark = SparkSession.builder \
    .appName('DataFrame') \
    .master('local[*]') \
    .getOrCreate()

In [2]:
spark.sparkContext

<SparkContext master=local[*] appName=DataFrame>

## Tworzenie DataFrame'u

### Plik

In [3]:
data_path = './SparkSQLdata/'

In [ ]:
# json usuniecie data_path+
people = spark.read.json(data_path+'people.json')

In [5]:
# json
people = spark.read.json(data_path+'people.json')

In [6]:
people.show()

+----+-------+
| age|   name|
+----+-------+
|null|Michael|
|  30|   Andy|
|  19| Justin|
|  35|   Emma|
|null|   null|
|  31|   null|
|  30|Michael|
|  19|    Eva|
|null|   Emma|
+----+-------+



In [12]:
# csv usuniecie data_path z people_txt = spark.read.option("inferSchema", "true").csv(data_path+'people.txt')
people_txt = spark.read.option("inferSchema", "true").csv('people.txt')

In [13]:
people_txt.show()

+-------+----+
|    _c0| _c1|
+-------+----+
|Michael|29.0|
|   Andy|30.0|
| Justin|19.0|
+-------+----+



### Kolekcja Row'ów

In [14]:
newPerson1 = Row(name='Greg', age=32)

In [15]:
newPerson1

Row(age=32, name='Greg')

In [16]:
newPerson1.name, newPerson1.age

('Greg', 32)

In [17]:
newPerson1['age']

32

In [18]:
'age' in newPerson1

True

In [19]:
newPerson = Row("age", "name")

In [20]:
newPerson2 = newPerson(24, 'Alice')

In [21]:
newPerson2

Row(age=24, name='Alice')

In [23]:
newPerson3 = newPerson(None, None)
newPerson4 = newPerson(33, None)
newPerson5 = newPerson(None, 'Peter')
newPerson6 = newPerson(32, 'Peter')
newPerson7 = newPerson(40, 'Greg')

In [24]:
newPeopleDF = spark.createDataFrame([newPerson1, newPerson2, newPerson3, newPerson4, 
                                     newPerson5, newPerson6, newPerson7])

In [25]:
newPeopleDF.show()

+----+-----+
| age| name|
+----+-----+
|  32| Greg|
|  24|Alice|
|null| null|
|  33| null|
|null|Peter|
|  32|Peter|
|  40| Greg|
+----+-----+



### Inne lokalne kolekcje

Typy danych: http://spark.apache.org/docs/latest/api/python/pyspark.sql.html#module-pyspark.sql.types

Kilka podstawowych: IntegerType, DoubleType, FloatType, StringType, BooleanType, NullType

In [26]:
from pyspark.sql.types import IntegerType, StringType, StructType, StructField

In [27]:
# definicja schematu
# StructType ~ Row
schema = StructType([StructField("V1", IntegerType()),
                     StructField("V2", StringType())])

In [28]:
# lokalna kolekcja - lista list
df = spark.createDataFrame([[1,2],[3,4]], schema)

In [29]:
df.show()

+---+---+
| V1| V2|
+---+---+
|  1|  2|
|  3|  4|
+---+---+



In [31]:
df.printSchema()

root
 |-- V1: integer (nullable = true)
 |-- V2: string (nullable = true)



### RDD

Przechodzenie RDD <-> DF

In [32]:
type(people)

pyspark.sql.dataframe.DataFrame

In [33]:
people.show()

+----+-------+
| age|   name|
+----+-------+
|null|Michael|
|  30|   Andy|
|  19| Justin|
|  35|   Emma|
|null|   null|
|  31|   null|
|  30|Michael|
|  19|    Eva|
|null|   Emma|
+----+-------+



DF -> RDD

In [34]:
type(people.rdd)

pyspark.rdd.RDD

In [35]:
people_rdd = people.rdd

In [36]:
people_rdd.collect()

[Row(age=None, name='Michael'),
 Row(age=30, name='Andy'),
 Row(age=19, name='Justin'),
 Row(age=35, name='Emma'),
 Row(age=None, name=None),
 Row(age=31, name=None),
 Row(age=30, name='Michael'),
 Row(age=19, name='Eva'),
 Row(age=None, name='Emma')]

In [37]:
people.rdd.collect()

[Row(age=None, name='Michael'),
 Row(age=30, name='Andy'),
 Row(age=19, name='Justin'),
 Row(age=35, name='Emma'),
 Row(age=None, name=None),
 Row(age=31, name=None),
 Row(age=30, name='Michael'),
 Row(age=19, name='Eva'),
 Row(age=None, name='Emma')]

In [39]:
people_rdd.map(tuple).collect()

[(None, 'Michael'),
 (30, 'Andy'),
 (19, 'Justin'),
 (35, 'Emma'),
 (None, None),
 (31, None),
 (30, 'Michael'),
 (19, 'Eva'),
 (None, 'Emma')]

RDD -> DF

In [40]:
people.rdd.toDF().show()

+----+-------+
| age|   name|
+----+-------+
|null|Michael|
|  30|   Andy|
|  19| Justin|
|  35|   Emma|
|null|   null|
|  31|   null|
|  30|Michael|
|  19|    Eva|
|null|   Emma|
+----+-------+



In [41]:
people.rdd.map(tuple).toDF().show()

+----+-------+
|  _1|     _2|
+----+-------+
|null|Michael|
|  30|   Andy|
|  19| Justin|
|  35|   Emma|
|null|   null|
|  31|   null|
|  30|Michael|
|  19|    Eva|
|null|   Emma|
+----+-------+



In [42]:
sc = spark.sparkContext

In [43]:
sc.parallelize([(x, x+1) for x in range(5)]).toDF().show()

+---+---+
| _1| _2|
+---+---+
|  0|  1|
|  1|  2|
|  2|  3|
|  3|  4|
|  4|  5|
+---+---+



In [44]:
# do toDF można podać schemat
schema = StructType([StructField("A", IntegerType()), StructField("B", StringType())])
sc.parallelize([(x, x+1) for x in range(5)]).toDF(schema).show()

+---+---+
|  A|  B|
+---+---+
|  0|  1|
|  1|  2|
|  2|  3|
|  3|  4|
|  4|  5|
+---+---+



In [46]:
sc.parallelize([(x, x+1) for x in range(5)]).toDF(schema).collect()

[Row(A=0, B='1'),
 Row(A=1, B='2'),
 Row(A=2, B='3'),
 Row(A=3, B='4'),
 Row(A=4, B='5')]

In [45]:
sc.parallelize([(x, x+1) for x in range(5)]).toDF(schema).collect()[0]

Row(A=0, B='1')

> **TODO**: Na dwa sposoby stwórz DF z 3 wierszami i 3 kolumnami - dwie typu string, jedna numeryczna

In [47]:
# DF z recznie podanych wartosci
# definicja schematu
# StructType ~ Row
schema_kolumna = StructType([StructField("kolumna1", IntegerType()),
                     StructField("kolumna2", StringType()),
                     StructField("kolumna3", StringType())])

In [50]:
# lokalna kolekcja - lista list
df_kolumna = spark.createDataFrame([[1,2,3],[4,5,6],[7,8,9]], schema_kolumna)

In [51]:
df_kolumna.show()

+--------+--------+--------+
|kolumna1|kolumna2|kolumna3|
+--------+--------+--------+
|       1|       2|       3|
|       4|       5|       6|
|       7|       8|       9|
+--------+--------+--------+



In [53]:
#RDD->DF
schema_kolumna2 = StructType([StructField("A", IntegerType()),
                              StructField("B", StringType()),
                              StructField("C", StringType())])
kz = sc.parallelize([(x, x+1,x+2) for x in range(5)]).toDF(schema_kolumna2)

In [54]:
kz.printSchema()

root
 |-- A: integer (nullable = true)
 |-- B: string (nullable = true)
 |-- C: string (nullable = true)



In [55]:
kz.show()


+---+---+---+
|  A|  B|  C|
+---+---+---+
|  0|  1|  2|
|  1|  2|  3|
|  2|  3|  4|
|  3|  4|  5|
|  4|  5|  6|
+---+---+---+



In [72]:
constr = Row("S1","S2","S3")
r1 = constr("a","b","1")
r2 = constr("a","b","2")
r3 = constr("a","b","3")
spark.createDataFrame([r1, r2, r3]).show()
spark.createDataFrame([r1, r2, r3]).printSchema()

+---+---+---+
| S1| S2| S3|
+---+---+---+
|  a|  b|  1|
|  a|  b|  2|
|  a|  b|  3|
+---+---+---+

root
 |-- S1: string (nullable = true)
 |-- S2: string (nullable = true)
 |-- S3: string (nullable = true)



> **TODO**: Z podanego RDD utwórz DF z nazwanymi kolumnami `name` i `age` oraz odpowiednimi typami (string i int)

In [59]:
myRDD = sc.textFile("people.txt")
myRDD.collect()

['Michael, 29', 'Andy, 30', 'Justin, 19']

In [60]:
schema = StructType([StructField("name", StringType()),
                    StructField("age", IntegerType())])

In [61]:
myRDD.map(lambda x: x.split(", ")).map(lambda x: (x[0], int(x[1])))\
.toDF(schema).show()

+-------+---+
|   name|age|
+-------+---+
|Michael| 29|
|   Andy| 30|
| Justin| 19|
+-------+---+



## Zapisywanie DataFrame'u

In [ ]:
#people.write.csv("...")

In [68]:
people.write.parquet("ppl_parquet")

In [71]:
spark.read.parquet("ppl_parquet").explain()

== Physical Plan ==
*FileScan parquet [age#257L,name#258] Batched: true, Format: Parquet, Location: InMemoryFileIndex[file:/home/mion/s/234/kzdunczy/Desktop/Spark/2.SQL/ppl_parquet], PartitionFilters: [], PushedFilters: [], ReadSchema: struct<age:bigint,name:string>


In [73]:
spark.read.parquet("ppl_parquet").select("age").explain()

== Physical Plan ==
*FileScan parquet [age#281L] Batched: true, Format: Parquet, Location: InMemoryFileIndex[file:/home/mion/s/234/kzdunczy/Desktop/Spark/2.SQL/ppl_parquet], PartitionFilters: [], PushedFilters: [], ReadSchema: struct<age:bigint>


****

## Praca z DataFrame'ami

### Kolumny

Odwolania do poszczegolnych kolumn

In [74]:
people.age

Column<b'age'>

In [75]:
people['age']

Column<b'age'>

In [76]:
people[0]

Column<b'age'>

Lista kolumn 

In [77]:
people.columns

['age', 'name']

In [78]:
people.dtypes

[('age', 'bigint'), ('name', 'string')]

Schemat

In [79]:
people.printSchema()

root
 |-- age: long (nullable = true)
 |-- name: string (nullable = true)



### Transformacje

##### select(cols)

Zwraca nowy DF ze wskazanymi kolumnami

In [80]:
people.select("age").show()

+----+
| age|
+----+
|null|
|  30|
|  19|
|  35|
|null|
|  31|
|  30|
|  19|
|null|
+----+



In [81]:
people.select("*").show()

+----+-------+
| age|   name|
+----+-------+
|null|Michael|
|  30|   Andy|
|  19| Justin|
|  35|   Emma|
|null|   null|
|  31|   null|
|  30|Michael|
|  19|    Eva|
|null|   Emma|
+----+-------+



> **TODO**: Wyświetl DF zawierający dwie kolumny `age` (people)

In [82]:
people.select("age","age").show()

+----+----+
| age| age|
+----+----+
|null|null|
|  30|  30|
|  19|  19|
|  35|  35|
|null|null|
|  31|  31|
|  30|  30|
|  19|  19|
|null|null|
+----+----+



##### drop(cols)

Zwraca nowy DF bez wskazanych kolumn

In [83]:
people.drop("age").show()

+-------+
|   name|
+-------+
|Michael|
|   Andy|
| Justin|
|   Emma|
|   null|
|   null|
|Michael|
|    Eva|
|   Emma|
+-------+



##### withColumn(name, column)

Zwraca nowy DF zawierający nowo zdefiniowaną kolumnę

In [86]:
people.withColumn("age+1", people.age + 1).show()

+----+-------+-----+
| age|   name|age+1|
+----+-------+-----+
|null|Michael| null|
|  30|   Andy|   31|
|  19| Justin|   20|
|  35|   Emma|   36|
|null|   null| null|
|  31|   null|   32|
|  30|Michael|   31|
|  19|    Eva|   20|
|null|   Emma| null|
+----+-------+-----+



> **TODO**: Wyświetl DF zawierający nową kolumnę z podwojonym wiekiem (people)

In [87]:
people.withColumn("twiceAge", people.age * 2).show()

+----+-------+--------+
| age|   name|twiceAge|
+----+-------+--------+
|null|Michael|    null|
|  30|   Andy|      60|
|  19| Justin|      38|
|  35|   Emma|      70|
|null|   null|    null|
|  31|   null|      62|
|  30|Michael|      60|
|  19|    Eva|      38|
|null|   Emma|    null|
+----+-------+--------+



##### withColumnRenamed(old, new)

Zwraca nowy DF ze zmienioną nazwą jednej kolumny

In [88]:
people.withColumnRenamed("age", "wiek").show()

+----+-------+
|wiek|   name|
+----+-------+
|null|Michael|
|  30|   Andy|
|  19| Justin|
|  35|   Emma|
|null|   null|
|  31|   null|
|  30|Michael|
|  19|    Eva|
|null|   Emma|
+----+-------+



> **TODO**: Wyświetl DF z kolumnami o nazwach `wiek` i `imię` (people)

In [89]:
people.withColumnRenamed("age", "wiek")\
.withColumnRenamed("wiek", "imię").show()

+----+-------+
|imię|   name|
+----+-------+
|null|Michael|
|  30|   Andy|
|  19| Justin|
|  35|   Emma|
|null|   null|
|  31|   null|
|  30|Michael|
|  19|    Eva|
|null|   Emma|
+----+-------+



##### filter(cond) / where(cond)

Zwraca nowy DF z wierszami spełniającymi zdefiniowany warunek

In [90]:
people.filter(people.age > 20).show()

+---+-------+
|age|   name|
+---+-------+
| 30|   Andy|
| 35|   Emma|
| 31|   null|
| 30|Michael|
+---+-------+



In [92]:
people.where(people.name.isNotNull()).show()

+---+-------+
|age|   name|
+---+-------+
| 30|   Andy|
| 19| Justin|
| 35|   Emma|
| 31|   null|
| 30|Michael|
| 19|    Eva|
+---+-------+



In [93]:
people.where(people.age.isNotNull()).show()

+---+-------+
|age|   name|
+---+-------+
| 30|   Andy|
| 19| Justin|
| 35|   Emma|
| 31|   null|
| 30|Michael|
| 19|    Eva|
+---+-------+



> **TODO**: Wyświetl DF z wierszami w których występuje mała lub wielka litera `a` (people)

In [96]:
people.filter((people.age > 20) & (people.age < 35)).show()

+---+-------+
|age|   name|
+---+-------+
| 30|   Andy|
| 31|   null|
| 30|Michael|
+---+-------+



In [108]:
people.where(people.name.contains("a") | people.name.contains("a")).show()

+----+-------+
| age|   name|
+----+-------+
|null|Michael|
|  35|   Emma|
|  30|Michael|
|  19|    Eva|
|null|   Emma|
+----+-------+



In [109]:
people.name.contains("a")

Column<b'contains(name, a)'>

In [110]:
people.dropna(how="all").show()

+----+-------+
| age|   name|
+----+-------+
|null|Michael|
|  30|   Andy|
|  19| Justin|
|  35|   Emma|
|  31|   null|
|  30|Michael|
|  19|    Eva|
|null|   Emma|
+----+-------+



##### dropna(how, thresh, subset)

Zwraca nowy DF z usuniętymi wierszami zawierającymi braki danych

In [111]:
people.dropna().show()

+---+-------+
|age|   name|
+---+-------+
| 30|   Andy|
| 19| Justin|
| 35|   Emma|
| 30|Michael|
| 19|    Eva|
+---+-------+



In [112]:
people.dropna(how="all").show()

+----+-------+
| age|   name|
+----+-------+
|null|Michael|
|  30|   Andy|
|  19| Justin|
|  35|   Emma|
|  31|   null|
|  30|Michael|
|  19|    Eva|
|null|   Emma|
+----+-------+



In [113]:
people.dropna(subset="name").show()

+----+-------+
| age|   name|
+----+-------+
|null|Michael|
|  30|   Andy|
|  19| Justin|
|  35|   Emma|
|  30|Michael|
|  19|    Eva|
|null|   Emma|
+----+-------+



> **TODO**: Wykorzystując parametr `thresh` wyświetl DF z wierszami w których występuje maksymalnie jeden brak danych (people)

In [116]:
people.dropna(thresh=1).show()

+----+-------+
| age|   name|
+----+-------+
|null|Michael|
|  30|   Andy|
|  19| Justin|
|  35|   Emma|
|  31|   null|
|  30|Michael|
|  19|    Eva|
|null|   Emma|
+----+-------+



##### fillna(value, subset)

Zwraca nowy DF z brakami danych zastąpionymi zdefiniowaną wartością/wartościami

In [117]:
people.fillna("X").show()

+----+-------+
| age|   name|
+----+-------+
|null|Michael|
|  30|   Andy|
|  19| Justin|
|  35|   Emma|
|null|      X|
|  31|      X|
|  30|Michael|
|  19|    Eva|
|null|   Emma|
+----+-------+



In [118]:
people.fillna({"age": 7, "name": "X"}).show()

+---+-------+
|age|   name|
+---+-------+
|  7|Michael|
| 30|   Andy|
| 19| Justin|
| 35|   Emma|
|  7|      X|
| 31|      X|
| 30|Michael|
| 19|    Eva|
|  7|   Emma|
+---+-------+



> **TODO**: Na podstawie kolumny `age` stwórz kolumnę `age_filled`, wykorzystując parametr `subset` wyświetl DF w którym braki danych w kolumnie `age_filled` zastąpione są wartością 27 (people)

In [133]:
people.withColumn("age_filled", people.age).fillna(11127, subset="age_filled").show()

+----+-------+----------+
| age|   name|age_filled|
+----+-------+----------+
|null|Michael|     11127|
|  30|   Andy|        30|
|  19| Justin|        19|
|  35|   Emma|        35|
|null|   null|     11127|
|  31|   null|        31|
|  30|Michael|        30|
|  19|    Eva|        19|
|null|   Emma|     11127|
+----+-------+----------+



In [131]:
people.dropna(subset="age").show()

+---+-------+
|age|   name|
+---+-------+
| 30|   Andy|
| 19| Justin|
| 35|   Emma|
| 31|   null|
| 30|Michael|
| 19|    Eva|
+---+-------+



##### replace(old, new, subset)

In [140]:
people.replace("Michael", "Mick").show()

+---+------+
|age|  name|
+---+------+
| 30|  Andy|
| 19|Justin|
| 35|  Emma|
| 31|  null|
| 30|  Mick|
| 19|   Eva|
+---+------+



In [141]:
people.replace("Michael", "Mick").where(people.age.isNotNull()).show()

+---+------+
|age|  name|
+---+------+
| 30|  Andy|
| 19|Justin|
| 35|  Emma|
| 31|  null|
| 30|  Mick|
| 19|   Eva|
+---+------+



In [135]:
people.replace({30: 31, 19: 22}).show()

+----+-------+
| age|   name|
+----+-------+
|null|Michael|
|  31|   Andy|
|  22| Justin|
|  35|   Emma|
|null|   null|
|  31|   null|
|  31|Michael|
|  22|    Eva|
|null|   Emma|
+----+-------+



> **TODO**: Na podstawie kolumny `name` stwórz kolumnę `surname`, wykorzystując parametr `subset` wyświetl DF w którym wartości w kolumnie `surname` zastąpione zostały w następujący sposób: Michael -> M, Andy -> A, itd. (people)

In [146]:
people.withColumn("surname", people.name).replace({"Michael": "M", "Andy": "A"}, subset="surname").show()

+----+-------+-------+
| age|   name|surname|
+----+-------+-------+
|null|Michael|      M|
|  30|   Andy|      A|
|  19| Justin| Justin|
|  35|   Emma|   Emma|
|null|   null|   null|
|  31|   null|   null|
|  30|Michael|      M|
|  19|    Eva|    Eva|
|null|   Emma|   Emma|
+----+-------+-------+



##### orderBy(cols)

In [147]:
people.orderBy("age").show()

+----+-------+
| age|   name|
+----+-------+
|null|Michael|
|null|   Emma|
|null|   null|
|  19| Justin|
|  19|    Eva|
|  30|Michael|
|  30|   Andy|
|  31|   null|
|  35|   Emma|
+----+-------+



In [148]:
people.orderBy(people.name.desc()).show()

+----+-------+
| age|   name|
+----+-------+
|null|Michael|
|  30|Michael|
|  19| Justin|
|  19|    Eva|
|  35|   Emma|
|null|   Emma|
|  30|   Andy|
|null|   null|
|  31|   null|
+----+-------+



In [149]:
people.orderBy(["age","name"], ascending=[0,1]).show()

+----+-------+
| age|   name|
+----+-------+
|  35|   Emma|
|  31|   null|
|  30|   Andy|
|  30|Michael|
|  19|    Eva|
|  19| Justin|
|null|   null|
|null|   Emma|
|null|Michael|
+----+-------+



> **TODO**: Wyświetl DF zawierający uporządkowane alfabetycznie imiona osób dla których dostępna jest informacja o wieku (people)

In [154]:
people.where(people.age.isNotNull()).orderBy(["name"], ascending=[1]).show()

+---+-------+
|age|   name|
+---+-------+
| 31|   null|
| 30|   Andy|
| 35|   Emma|
| 19|    Eva|
| 19| Justin|
| 30|Michael|
+---+-------+



In [156]:
people.dropna().orderBy(["name"]).select("name").show()

+-------+
|   name|
+-------+
|   Andy|
|   Emma|
|    Eva|
| Justin|
|Michael|
+-------+



##### union(df)

Zwraca nowy DF zawierający wszystkie wiersze z dwóch łączonych DFów (odpowiednik UNION ALL w SQL)

In [157]:
# union sam nie dopasowuje kolumn!
people.union(people_txt).show()

+-------+-------+
|    age|   name|
+-------+-------+
|   null|Michael|
|     30|   Andy|
|     19| Justin|
|     35|   Emma|
|   null|   null|
|     31|   null|
|     30|Michael|
|     19|    Eva|
|   null|   Emma|
|Michael|   29.0|
|   Andy|   30.0|
| Justin|   19.0|
+-------+-------+



In [158]:
people.union(people_txt.select(sorted(people_txt.columns, reverse=True))).show()

+----+-------+
| age|   name|
+----+-------+
|null|Michael|
|30.0|   Andy|
|19.0| Justin|
|35.0|   Emma|
|null|   null|
|31.0|   null|
|30.0|Michael|
|19.0|    Eva|
|null|   Emma|
|29.0|Michael|
|30.0|   Andy|
|19.0| Justin|
+----+-------+



In [159]:
# lub
people.select("name", "age").union(people_txt).show()

+-------+----+
|   name| age|
+-------+----+
|Michael|null|
|   Andy|30.0|
| Justin|19.0|
|   Emma|35.0|
|   null|null|
|   null|31.0|
|Michael|30.0|
|    Eva|19.0|
|   Emma|null|
|Michael|29.0|
|   Andy|30.0|
| Justin|19.0|
+-------+----+



##### intersect(df)

Zwraca nowy DF zawierający jedynie wiersze występujące w obydwóch DFach

In [160]:
people.select("name", "age").intersect(people_txt).show()

+------+----+
|  name| age|
+------+----+
|Justin|19.0|
|  Andy|30.0|
+------+----+



##### subtract(df)

Zwraca nowy DF z wierszami z oryginalnego DF które nie występują w drugim DF

In [161]:
people.select("name", "age").subtract(people_txt).show()

+-------+----+
|   name| age|
+-------+----+
|Michael|30.0|
|   null|null|
|   Emma|35.0|
|    Eva|19.0|
|Michael|null|
|   null|31.0|
|   Emma|null|
+-------+----+



##### distinct

Zwraca nowy DF z unikalnymi wierszami z oryginalnego DF

In [162]:
people.select("name", "age").union(people_txt).distinct().show()

+-------+----+
|   name| age|
+-------+----+
|Michael|30.0|
|   null|null|
|   Emma|35.0|
|    Eva|19.0|
|Michael|null|
|Michael|29.0|
|   null|31.0|
| Justin|19.0|
|   Emma|null|
|   Andy|30.0|
+-------+----+



##### dropDuplicates(subset)

Zwraca nowy DF z unikalnymi wierszami z oryginalnego DF (na podstawie całych wierszy lub wskazanych kolumn)

In [163]:
people.select("name", "age").union(people_txt).dropDuplicates().show()

+-------+----+
|   name| age|
+-------+----+
|Michael|30.0|
|   null|null|
|   Emma|35.0|
|    Eva|19.0|
|Michael|null|
|Michael|29.0|
|   null|31.0|
| Justin|19.0|
|   Emma|null|
|   Andy|30.0|
+-------+----+



In [164]:
people.dropDuplicates(subset=["name"]).show()

+----+-------+
| age|   name|
+----+-------+
|null|   null|
|  19|    Eva|
|null|Michael|
|  35|   Emma|
|  30|   Andy|
|  19| Justin|
+----+-------+



##### join(df, on, how)

Zwraca nowy DF powstały na podstawie połączenia dwóch DFów w oparciu o wartości we wskazanej kolumnie/kolumnach

In [165]:
people.join(newPeopleDF, on="name", how="outer").show()

+-------+----+----+
|   name| age| age|
+-------+----+----+
|   Greg|null|  32|
|   Greg|null|  40|
|   null|null|null|
|   null|  31|null|
|   null|null|null|
|   null|null|  33|
|    Eva|  19|null|
|Michael|null|null|
|Michael|  30|null|
|  Alice|null|  24|
|   Emma|  35|null|
|   Emma|null|null|
|   Andy|  30|null|
| Justin|  19|null|
|  Peter|null|null|
|  Peter|null|  32|
+-------+----+----+



In [166]:
people.join(newPeopleDF, on="name").show()

+----+---+---+
|name|age|age|
+----+---+---+
+----+---+---+



In [167]:
people.join(people_txt, people.name == people_txt._c0).show()

+----+-------+-------+----+
| age|   name|    _c0| _c1|
+----+-------+-------+----+
|null|Michael|Michael|29.0|
|  30|   Andy|   Andy|30.0|
|  19| Justin| Justin|19.0|
|  30|Michael|Michael|29.0|
+----+-------+-------+----+



> **Uwaga**
>
>Join to popularna, ale kosztowna operacja.<br>
>W sytuacji, kiedy jeden z łaczonych DataFramow jest znacznie mniejszy (w szczegolnosci na tyle mały, że w całości mieści się w pamięci), zaleca sie zastosowanie *broadcast hash join*.<br>
>(Mała tabela zostanie zebrana do pamięci i wysłana do każdego noda).<br>
>W niektórych przypadkach optymalizator sam za nas zdecyduje o zastosowaniu *broadcast hash join*.

In [168]:
from pyspark.sql.functions import broadcast
newPeopleDF.join(broadcast(spark.createDataFrame([Row(age=20, name='Greg')])), on='name').show()

+----+---+---+
|name|age|age|
+----+---+---+
|Greg| 32| 20|
|Greg| 40| 20|
+----+---+---+



##### groupBy(cols)

Zwraca nowy DF pogrupowany po wskazanej kolumnie/kolumnach (nie jest to typowy DF, nie można go podejrzeć używając show)

In [169]:
people.groupBy("name")

In [170]:
people.groupBy("name").count().show()

+-------+-----+
|   name|count|
+-------+-----+
|   null|    2|
|    Eva|    1|
|Michael|    2|
|   Emma|    2|
|   Andy|    1|
| Justin|    1|
+-------+-----+



In [171]:
# mean/max/min
people.groupBy("name").max().show()

+-------+--------+
|   name|max(age)|
+-------+--------+
|   null|      31|
|    Eva|      19|
|Michael|      30|
|   Emma|      35|
|   Andy|      30|
| Justin|      19|
+-------+--------+



In [172]:
people.groupBy("name").max("age").show()

+-------+--------+
|   name|max(age)|
+-------+--------+
|   null|      31|
|    Eva|      19|
|Michael|      30|
|   Emma|      35|
|   Andy|      30|
| Justin|      19|
+-------+--------+



> **TODO**: Wyświetl DF z informacją ile razy pojawił się każdy wiek (people)

In [177]:
people.groupBy("age").count().show()

+----+-----+
| age|count|
+----+-----+
|  19|    2|
|  31|    1|
|null|    3|
|  35|    1|
|  30|    2|
+----+-----+



##### agg(expres)

Zwraca nowy DF na powstały w wyniku zastosowania wskazanych agregacji

In [178]:
import pyspark.sql.functions as f

In [179]:
people.groupBy("name").agg(f.max("age")).show()

+-------+--------+
|   name|max(age)|
+-------+--------+
|   null|      31|
|    Eva|      19|
|Michael|      30|
|   Emma|      35|
|   Andy|      30|
| Justin|      19|
+-------+--------+



In [183]:
people.groupBy("name").agg(f.max("age"), 
                           f.count("name"), 
                           f.count("age")).show()

+-------+--------+-----------+----------+
|   name|max(age)|count(name)|count(age)|
+-------+--------+-----------+----------+
|   null|      31|          0|         1|
|    Eva|      19|          1|         1|
|Michael|      30|          2|         1|
|   Emma|      35|          2|         1|
|   Andy|      30|          1|         1|
| Justin|      19|          1|         1|
+-------+--------+-----------+----------+



In [184]:
people.groupBy("name").agg(f.max("age").alias("max_age")).show()

+-------+-------+
|   name|max_age|
+-------+-------+
|   null|     31|
|    Eva|     19|
|Michael|     30|
|   Emma|     35|
|   Andy|     30|
| Justin|     19|
+-------+-------+



In [185]:
people.agg(f.max("age")).show()

+--------+
|max(age)|
+--------+
|      35|
+--------+



##### describe(cols)

Zwraca nowy DF zawierający podstawowe statystyki wszystkich lub wskazanych kolumn

In [186]:
people.describe().show()

+-------+------------------+-------+
|summary|               age|   name|
+-------+------------------+-------+
|  count|                 6|      7|
|   mean|27.333333333333332|   null|
| stddev| 6.713171133426189|   null|
|    min|                19|   Andy|
|    max|                35|Michael|
+-------+------------------+-------+



In [187]:
people.describe("name").show()

+-------+-------+
|summary|   name|
+-------+-------+
|  count|      7|
|   mean|   null|
| stddev|   null|
|    min|   Andy|
|    max|Michael|
+-------+-------+



### Akcje

##### show(n, truncate, vertical)

Wyświetla n wierszy DF

In [188]:
people.show()

+----+-------+
| age|   name|
+----+-------+
|null|Michael|
|  30|   Andy|
|  19| Justin|
|  35|   Emma|
|null|   null|
|  31|   null|
|  30|Michael|
|  19|    Eva|
|null|   Emma|
+----+-------+



In [189]:
people.show(2)

+----+-------+
| age|   name|
+----+-------+
|null|Michael|
|  30|   Andy|
+----+-------+
only showing top 2 rows



In [190]:
people.withColumn("long", f.lit("xxxxxxxxxxxxx xxxxxxxxxxxxxx xxxxxxxxxxxxxxx")).show(4)
people.withColumn("long", f.lit("xxxxxxxxxxxxx xxxxxxxxxxxxxx xxxxxxxxxxxxxxx")).show(4, truncate=False)

+----+-------+--------------------+
| age|   name|                long|
+----+-------+--------------------+
|null|Michael|xxxxxxxxxxxxx xxx...|
|  30|   Andy|xxxxxxxxxxxxx xxx...|
|  19| Justin|xxxxxxxxxxxxx xxx...|
|  35|   Emma|xxxxxxxxxxxxx xxx...|
+----+-------+--------------------+
only showing top 4 rows

+----+-------+--------------------------------------------+
|age |name   |long                                        |
+----+-------+--------------------------------------------+
|null|Michael|xxxxxxxxxxxxx xxxxxxxxxxxxxx xxxxxxxxxxxxxxx|
|30  |Andy   |xxxxxxxxxxxxx xxxxxxxxxxxxxx xxxxxxxxxxxxxxx|
|19  |Justin |xxxxxxxxxxxxx xxxxxxxxxxxxxx xxxxxxxxxxxxxxx|
|35  |Emma   |xxxxxxxxxxxxx xxxxxxxxxxxxxx xxxxxxxxxxxxxxx|
+----+-------+--------------------------------------------+
only showing top 4 rows



In [192]:
people.show()

+----+-------+
| age|   name|
+----+-------+
|null|Michael|
|  30|   Andy|
|  19| Justin|
|  35|   Emma|
|null|   null|
|  31|   null|
|  30|Michael|
|  19|    Eva|
|null|   Emma|
+----+-------+



##### collect 

Zwraca DF jako lokalną kolekcję (listę) wierszy 

**NIEBEZPIECZNE!**

In [193]:
people.collect()

[Row(age=None, name='Michael'),
 Row(age=30, name='Andy'),
 Row(age=19, name='Justin'),
 Row(age=35, name='Emma'),
 Row(age=None, name=None),
 Row(age=31, name=None),
 Row(age=30, name='Michael'),
 Row(age=19, name='Eva'),
 Row(age=None, name='Emma')]

In [194]:
people.limit(4).collect()

[Row(age=None, name='Michael'),
 Row(age=30, name='Andy'),
 Row(age=19, name='Justin'),
 Row(age=35, name='Emma')]

##### take(n) / head(n)

Zwraca piersze n wierszy z DF jako lokalną kolekcję (listę) wierszy

**NIEBEZPIECZNE!**

In [195]:
people.take(5)

[Row(age=None, name='Michael'),
 Row(age=30, name='Andy'),
 Row(age=19, name='Justin'),
 Row(age=35, name='Emma'),
 Row(age=None, name=None)]

In [196]:
people.head(5)

[Row(age=None, name='Michael'),
 Row(age=30, name='Andy'),
 Row(age=19, name='Justin'),
 Row(age=35, name='Emma'),
 Row(age=None, name=None)]

In [197]:
people.take(1)

[Row(age=None, name='Michael')]

In [198]:
people.head()

Row(age=None, name='Michael')

##### first

Zwraca pierwszy wiersz z DF

**NIEBEZPIECZNE!**

In [199]:
people.first()

Row(age=None, name='Michael')

##### count

Zlicza liczbę wierszy w DF

In [200]:
people.count()

9

##### toPandas

Zwraca DF jako Pandas DF

**NIEBEZPIECZNE!**

In [201]:
people.toPandas()

,age,name
0,NaN,Michael
1,30.0,Andy
2,19.0,Justin
3,35.0,Emma
4,NaN,None
5,31.0,None
6,30.0,Michael
7,19.0,Eva
8,NaN,Emma


### Funkcje SQL

Z funkcji tych można korzystać wewnątrz operacji: select, withColumn, agg

In [202]:
import pyspark.sql.functions as f

##### max, min, avg, stddv

In [203]:
people.select(f.max(people.age).alias("max"), 
              f.min(people.age).alias("min"), 
              f.avg(people.age).alias("avg"), 
              f.stddev(people.age).alias("stddev"), 
              f.count(people.age).alias("count")).show()

+---+---+------------------+-----------------+-----+
|max|min|               avg|           stddev|count|
+---+---+------------------+-----------------+-----+
| 35| 19|27.333333333333332|6.713171133426189|    6|
+---+---+------------------+-----------------+-----+



In [204]:
people.agg(f.max(people.age).alias("max"), 
           f.min(people.age).alias("min"), 
           f.avg(people.age).alias("avg"), 
           f.stddev(people.age).alias("stddev"), 
           f.count(people.age).alias("count")).show()

+---+---+------------------+-----------------+-----+
|max|min|               avg|           stddev|count|
+---+---+------------------+-----------------+-----+
| 35| 19|27.333333333333332|6.713171133426189|    6|
+---+---+------------------+-----------------+-----+



##### lit

Tworzy kolumnę ze stałą wartością

In [205]:
people.withColumn("const", f.lit(7)).show()

+----+-------+-----+
| age|   name|const|
+----+-------+-----+
|null|Michael|    7|
|  30|   Andy|    7|
|  19| Justin|    7|
|  35|   Emma|    7|
|null|   null|    7|
|  31|   null|    7|
|  30|Michael|    7|
|  19|    Eva|    7|
|null|   Emma|    7|
+----+-------+-----+



##### asc/desc

Pozwalają na zdefiniowanie porządek sortowania w orderBy

In [206]:
people.orderBy(f.asc("age")).show()

+----+-------+
| age|   name|
+----+-------+
|null|Michael|
|null|   Emma|
|null|   null|
|  19| Justin|
|  19|    Eva|
|  30|Michael|
|  30|   Andy|
|  31|   null|
|  35|   Emma|
+----+-------+



In [207]:
people.orderBy(f.desc("age")).show()

+----+-------+
| age|   name|
+----+-------+
|  35|   Emma|
|  31|   null|
|  30|Michael|
|  30|   Andy|
|  19| Justin|
|  19|    Eva|
|null|Michael|
|null|   null|
|null|   Emma|
+----+-------+



##### col

Pozwala na odwołanie się do kolumny

In [208]:
people.select(f.col("age")).show()

+----+
| age|
+----+
|null|
|  30|
|  19|
|  35|
|null|
|  31|
|  30|
|  19|
|null|
+----+



In [211]:
# do operacji na kolumnach potrzebne jest odwołanie do kolumny - nazwa nie wystarcza
people.withColumn("X", f.lit(3))\
.withColumn("ageX", "X" * "age").show()

TypeError: can't multiply sequence by non-int of type 'str'

In [214]:
# DF people nie zawiera kolumny X
people.withColumn("X", f.lit(3))\
.withColumn("ageX", people["X"] * f.col("age")).show()

AnalysisException: 'Cannot resolve column name "X" among (age, name);'

In [212]:
# DF people nie zawiera kolumny X
people.withColumn("X", f.lit(3))\
.withColumn("ageX", people["X"] * people["age"]).show()

AnalysisException: 'Cannot resolve column name "X" among (age, name);'

In [215]:
people.withColumn("X", f.lit(3)).withColumn("ageX", f.col("X") * f.col("age")).show()

+----+-------+---+----+
| age|   name|  X|ageX|
+----+-------+---+----+
|null|Michael|  3|null|
|  30|   Andy|  3|  90|
|  19| Justin|  3|  57|
|  35|   Emma|  3| 105|
|null|   null|  3|null|
|  31|   null|  3|  93|
|  30|Michael|  3|  90|
|  19|    Eva|  3|  57|
|null|   Emma|  3|null|
+----+-------+---+----+



------------------------------------------------------------

In [216]:
t_df = spark.createDataFrame([('2019-04-27 09:00:00',), ('2019-06-01 16:30:00',)], ['time'])
t_df.show()
t_df.dtypes

+-------------------+
|               time|
+-------------------+
|2019-04-27 09:00:00|
|2019-06-01 16:30:00|
+-------------------+



[('time', 'string')]

----------------------------------------------------

##### split(str, pattern)

Dzieli string na podstawie wzorca (regex)

In [217]:
split_df = t_df.select(f.split("time", " ").alias("splitted"))
split_df.show(truncate=False)

+----------------------+
|splitted              |
+----------------------+
|[2019-04-27, 09:00:00]|
|[2019-06-01, 16:30:00]|
+----------------------+



In [218]:
# dwa sposoby na wyciąganie wartości (tworzenie nowych kolumn) z kolumny zawierającej array'e
split_df.select(split_df.splitted[0].alias("date"), split_df.splitted.getItem(1).alias("time")).show()

+----------+--------+
|      date|    time|
+----------+--------+
|2019-04-27|09:00:00|
|2019-06-01|16:30:00|
+----------+--------+



##### explode

Tworzy nowy wiersz z każdego elementu arraya

In [219]:
split_df.dtypes

[('splitted', 'array<string>')]

In [220]:
split_df.select(f.explode("splitted").alias("exploded")).show()

+----------+
|  exploded|
+----------+
|2019-04-27|
|  09:00:00|
|2019-06-01|
|  16:30:00|
+----------+



##### from_utc_timestamp(timestamp, tz)

In [221]:
t_df.withColumn("time", f.from_utc_timestamp(t_df.time, "GMT")).show()
t_df.withColumn("time", f.from_utc_timestamp(t_df.time, "GMT")).dtypes

+-------------------+
|               time|
+-------------------+
|2019-04-27 09:00:00|
|2019-06-01 16:30:00|
+-------------------+



[('time', 'timestamp')]

##### lower, upper

In [222]:
people.select(f.lower(people.name), f.upper(people.name)).show()

+-----------+-----------+
|lower(name)|upper(name)|
+-----------+-----------+
|    michael|    MICHAEL|
|       andy|       ANDY|
|     justin|     JUSTIN|
|       emma|       EMMA|
|       null|       null|
|       null|       null|
|    michael|    MICHAEL|
|        eva|        EVA|
|       emma|       EMMA|
+-----------+-----------+



##### length

In [223]:
people.select("name", f.length(f.col("name"))).show()

+-------+------------+
|   name|length(name)|
+-------+------------+
|Michael|           7|
|   Andy|           4|
| Justin|           6|
|   Emma|           4|
|   null|        null|
|   null|        null|
|Michael|           7|
|    Eva|           3|
|   Emma|           4|
+-------+------------+



##### substring(str, pos, len)

In [224]:
people.select(f.substring("name", 1, 2).alias("first_letters")).show()

+-------------+
|first_letters|
+-------------+
|           Mi|
|           An|
|           Ju|
|           Em|
|         null|
|         null|
|           Mi|
|           Ev|
|           Em|
+-------------+



> **TODO**: Wyświetl DF z kolumną zawierającą ostatnie dwie litery imion (people)

In [227]:
people.select(f.substring("name", -2, 2).alias("first_letters")).show()

+-------------+
|first_letters|
+-------------+
|           el|
|           dy|
|           in|
|           ma|
|         null|
|         null|
|           el|
|           va|
|           ma|
+-------------+



##### rand(seed), randn(seed)

Tworzą kolumny z losowymi wartościami z rozkładów odpowiednio: jednostajnego (0-1), normalnego (0,1)

In [229]:
people.select("name", f.rand(42), f.rand(42),
              f.rand(1), f.randn(7)).show()

+-------+--------------------+--------------------+-------------------+--------------------+
|   name|            rand(42)|            rand(42)|            rand(1)|            randn(7)|
+-------+--------------------+--------------------+-------------------+--------------------+
|Michael|  0.6661236774413726|  0.6661236774413726|0.13385709732307427| -1.2904230199480902|
|   Andy|  0.8583151351252906|  0.8583151351252906| 0.5897562959687032|  1.0028842205245463|
| Justin|  0.9139963682495181|  0.9139963682495181|0.01540012100242305| -1.2023777869506285|
|   Emma|  0.8664942556157945|  0.8664942556157945|0.22569943461197162| -1.3767992982903736|
|   null| 0.43003259523546167| 0.43003259523546167| 0.9207602095112212|-0.19014010351113017|
|   null|  0.5578165506555315|  0.5578165506555315| 0.6222816020094926| -0.8628423790295872|
|Michael|0.003566841203428539|0.003566841203428539| 0.1029837279488438|  0.6120130788148804|
|    Eva| 0.12234058267065306| 0.12234058267065306| 0.6678762139023474

In [ ]:
people.select("name", f.rand(42), f.rand(42),
              f.rand(1), f.randn(7)).show()

In [ ]:
import random gauss

In [231]:

people.select("name", f.round(f.rand(42), 2)\
              .alias("r1"), f.lit(1) * random.gauss(0,1).show()

SyntaxError: unexpected EOF while parsing (<ipython-input-231-ca1a5293756d>, line 2)

##### when(cond, value)

In [232]:
people.select("name", f.when(people.name.startswith("M"), "X").otherwise("Y").alias("when")).show()

+-------+----+
|   name|when|
+-------+----+
|Michael|   X|
|   Andy|   Y|
| Justin|   Y|
|   Emma|   Y|
|   null|   Y|
|   null|   Y|
|Michael|   X|
|    Eva|   Y|
|   Emma|   Y|
+-------+----+



In [233]:
people.select("name", 
              f.when(people.name.startswith("M"), "X")\
              .when(people.name.startswith("E"), "Z")\
              .otherwise("Y").alias("when")).show()

+-------+----+
|   name|when|
+-------+----+
|Michael|   X|
|   Andy|   Y|
| Justin|   Y|
|   Emma|   Z|
|   null|   Y|
|   null|   Y|
|Michael|   X|
|    Eva|   Z|
|   Emma|   Z|
+-------+----+



> **TODO**: Wyświetl DF z kolumną zawierającą wartość `>4` gdy imię ma więcej niż 4 litery oraz `<=4` w przeciwnym przypadku (people)

In [291]:
people.select("name", f.when(f.length(f.col("name")) > 4 , ">4")\
              .when(f.when(f.length(f.col("name")) <= 4 , "<=4")))
              .otherwise("None").alias("flag").show()

IndentationError: unexpected indent (<ipython-input-291-eb737206d9bb>, line 2)

-------------------

In [248]:
import random
random.seed(123)

years = 6
names = ['Alice', 'Betty', 'Chris', 'Dan', 'Greg']
names_count = len(names)
names = sorted(names*years)
year = [y for y in range(2005, 2005+years)] * names_count
starting_salary = [round(random.gauss(4000, 1000),2) for i in range(names_count)]
salary = [0 for i in range(years * names_count)]
salary[::years] = starting_salary
for n in range(names_count):
    for y in range(years-1):
        index = (years*n+1)+y
        salary[index] = round(salary[index-1]*(1+random.gauss(0.1,0.09)),2)

In [249]:
salaryHistory = spark.createDataFrame([Row(name=n, year=y, salary=s) for n,y,s in zip(names, year, salary)])
salaryHistory = salaryHistory.filter((salaryHistory['name'] != 'Greg') | (salaryHistory['year'] != 2006))
salaryHistory = salaryHistory.union(spark.createDataFrame([Row('Alice', 3000, 2005)])).orderBy("name","year")

In [250]:
salaryHistory.show(30)

+-----+-------+----+
| name| salary|year|
+-----+-------+----+
|Alice|4404.23|2005|
|Alice| 3000.0|2005|
|Alice|4780.34|2006|
|Alice|4881.72|2007|
|Alice|5280.86|2008|
|Alice|5976.68|2009|
|Alice|6320.14|2010|
|Betty|4138.01|2005|
|Betty|4376.94|2006|
|Betty|5117.68|2007|
|Betty|5630.26|2008|
|Betty|6222.57|2009|
|Betty|6623.97|2010|
|Chris|3601.42|2005|
|Chris|4015.66|2006|
|Chris|4304.73|2007|
|Chris|4650.33|2008|
|Chris|4932.86|2009|
|Chris|5869.98|2010|
|  Dan|4262.75|2005|
|  Dan|4829.28|2006|
|  Dan|5204.81|2007|
|  Dan|6244.11|2008|
|  Dan|7642.18|2009|
|  Dan|9906.35|2010|
| Greg|4226.89|2005|
| Greg|4935.72|2007|
| Greg|5274.04|2008|
| Greg|6227.36|2009|
| Greg|6033.33|2010|
+-----+-------+----+



---------------------

### Funkcje okienne (window functions)

Służą do obliczania agregowanych wartości w grupach definiowanych oknem (window).<br>
Zwracają wiele rekordow (tyle ile na wejsciu w grupie).

In [251]:
from pyspark.sql.window import Window

##### partitionBy(cols)

Konstruktor tworzący okna - podział DF ze względu na wartości w podanej kolumnie/kolumnach

funkcja.over(definicja okna)

In [252]:
# definicja 'okna'
windowSpec = Window.partitionBy('name')

salaryHistory.withColumn('nameCount', f.count(salaryHistory['name']).over(windowSpec)).show(30)

+-----+-------+----+---------+
| name| salary|year|nameCount|
+-----+-------+----+---------+
|Chris|3601.42|2005|        6|
|Chris|4015.66|2006|        6|
|Chris|4304.73|2007|        6|
|Chris|4650.33|2008|        6|
|Chris|4932.86|2009|        6|
|Chris|5869.98|2010|        6|
| Greg|4226.89|2005|        5|
| Greg|4935.72|2007|        5|
| Greg|5274.04|2008|        5|
| Greg|6227.36|2009|        5|
| Greg|6033.33|2010|        5|
|Betty|4138.01|2005|        6|
|Betty|4376.94|2006|        6|
|Betty|5117.68|2007|        6|
|Betty|5630.26|2008|        6|
|Betty|6222.57|2009|        6|
|Betty|6623.97|2010|        6|
|  Dan|4262.75|2005|        6|
|  Dan|4829.28|2006|        6|
|  Dan|5204.81|2007|        6|
|  Dan|6244.11|2008|        6|
|  Dan|7642.18|2009|        6|
|  Dan|9906.35|2010|        6|
|Alice|4404.23|2005|        7|
|Alice| 3000.0|2005|        7|
|Alice|4780.34|2006|        7|
|Alice|4881.72|2007|        7|
|Alice|5280.86|2008|        7|
|Alice|5976.68|2009|        7|
|Alice|6

In [254]:
salaryHistory.groupBy("name").count().show(30)

+-----+-----+
| name|count|
+-----+-----+
|Chris|    6|
| Greg|    5|
|Betty|    6|
|  Dan|    6|
|Alice|    7|
+-----+-----+



In [ ]:
salaryHistory.join(salaryHistory)

> **TODO**: Wyświetl DF z kolumną zawierającą różnicę między pensją danej osoby w konkretnym roku a średnią pensją danej osoby w analizowanym okresie (salaryHistory)

In [279]:
# definicja 'okna'
windowSpec = Window.partitionBy('name')

salaryHistory.withColumn('avg', f.avg(salaryHistory['salary']).over(windowSpec))\
.withColumn("diff", f.col("salary") - f.col("avg")).show(30)

+-----+-------+----+-----------------+-------------------+
| name| salary|year|              avg|               diff|
+-----+-------+----+-----------------+-------------------+
|Chris|3601.42|2005|4562.496666666667| -961.0766666666668|
|Chris|4015.66|2006|4562.496666666667|  -546.836666666667|
|Chris|4304.73|2007|4562.496666666667|-257.76666666666733|
|Chris|4650.33|2008|4562.496666666667|  87.83333333333303|
|Chris|4932.86|2009|4562.496666666667|  370.3633333333328|
|Chris|5869.98|2010|4562.496666666667| 1307.4833333333327|
| Greg|4226.89|2005|5339.468000000001|-1112.5780000000004|
| Greg|4935.72|2007|5339.468000000001| -403.7480000000005|
| Greg|5274.04|2008|5339.468000000001|  -65.4280000000008|
| Greg|6227.36|2009|5339.468000000001|  887.8919999999989|
| Greg|6033.33|2010|5339.468000000001|  693.8619999999992|
|Betty|4138.01|2005|5351.571666666667|-1213.5616666666665|
|Betty|4376.94|2006|5351.571666666667| -974.6316666666671|
|Betty|5117.68|2007|5351.571666666667|-233.8916666666664

##### orderBy(cols)

Definiuje wewnątrz każdego okna sortowanie w oparciu o wskazaną kolumnę/kolumny

In [278]:
windowSpec = Window.partitionBy("name").orderBy("year")

salaryHistory.withColumn("rank", f.rank().over(windowSpec))\
             .withColumn("dense_rank", f.dense_rank().over(windowSpec))\
             .withColumn("row_num", f.row_number().over(windowSpec)).show(30)

+-----+-------+----+----+----------+-------+
| name| salary|year|rank|dense_rank|row_num|
+-----+-------+----+----+----------+-------+
|Chris|3601.42|2005|   1|         1|      1|
|Chris|4015.66|2006|   2|         2|      2|
|Chris|4304.73|2007|   3|         3|      3|
|Chris|4650.33|2008|   4|         4|      4|
|Chris|4932.86|2009|   5|         5|      5|
|Chris|5869.98|2010|   6|         6|      6|
| Greg|4226.89|2005|   1|         1|      1|
| Greg|4935.72|2007|   2|         2|      2|
| Greg|5274.04|2008|   3|         3|      3|
| Greg|6227.36|2009|   4|         4|      4|
| Greg|6033.33|2010|   5|         5|      5|
|Betty|4138.01|2005|   1|         1|      1|
|Betty|4376.94|2006|   2|         2|      2|
|Betty|5117.68|2007|   3|         3|      3|
|Betty|5630.26|2008|   4|         4|      4|
|Betty|6222.57|2009|   5|         5|      5|
|Betty|6623.97|2010|   6|         6|      6|
|  Dan|4262.75|2005|   1|         1|      1|
|  Dan|4829.28|2006|   2|         2|      2|
|  Dan|520

> **TODO**: Wyświetl DF z kolumną zawierającą różnicę w pensji rok do roku dla każdej z osób - wykorzystaj funkcję `lag` (salaryHistory)

In [286]:
windowSpec = Window.partitionBy("name").orderBy("year")

salaryHistory\
.withColumn("lag", f.lag(f.col("salary")).over(windowSpec))\
.withColumn("diff", f.col("salary") - f.col("lag")).show(30)

+-----+-------+----+-------+-------------------+
| name| salary|year|    lag|               diff|
+-----+-------+----+-------+-------------------+
|Chris|3601.42|2005|   null|               null|
|Chris|4015.66|2006|3601.42|  414.2399999999998|
|Chris|4304.73|2007|4015.66|  289.0699999999997|
|Chris|4650.33|2008|4304.73| 345.60000000000036|
|Chris|4932.86|2009|4650.33| 282.52999999999975|
|Chris|5869.98|2010|4932.86|  937.1199999999999|
| Greg|4226.89|2005|   null|               null|
| Greg|4935.72|2007|4226.89|  708.8299999999999|
| Greg|5274.04|2008|4935.72|  338.3199999999997|
| Greg|6227.36|2009|5274.04|  953.3199999999997|
| Greg|6033.33|2010|6227.36|-194.02999999999975|
|Betty|4138.01|2005|   null|               null|
|Betty|4376.94|2006|4138.01| 238.92999999999938|
|Betty|5117.68|2007|4376.94|  740.7400000000007|
|Betty|5630.26|2008|5117.68|  512.5799999999999|
|Betty|6222.57|2009|5630.26|  592.3099999999995|
|Betty|6623.97|2010|6222.57| 401.40000000000055|
|  Dan|4262.75|2005|

##### rangeBetween(start, end)

Dodaje wewnątrz każdego okna zakres (offset) na którym zastosowana zostanie wskazana funkcja (remisy w sortowaniu traktuje jak funkcja rank)

In [287]:
windowSpec = Window.partitionBy("name").orderBy("year").rangeBetween(Window.unboundedPreceding,0)

salaryHistory.withColumn("moving_avg", f.avg(f.col("salary")).over(windowSpec)).show(30)

+-----+-------+----+------------------+
| name| salary|year|        moving_avg|
+-----+-------+----+------------------+
|Chris|3601.42|2005|           3601.42|
|Chris|4015.66|2006|           3808.54|
|Chris|4304.73|2007|3973.9366666666665|
|Chris|4650.33|2008|          4143.035|
|Chris|4932.86|2009|            4301.0|
|Chris|5869.98|2010| 4562.496666666667|
| Greg|4226.89|2005|           4226.89|
| Greg|4935.72|2007|          4581.305|
| Greg|5274.04|2008| 4812.216666666667|
| Greg|6227.36|2009|5166.0025000000005|
| Greg|6033.33|2010| 5339.468000000001|
|Betty|4138.01|2005|           4138.01|
|Betty|4376.94|2006|          4257.475|
|Betty|5117.68|2007|           4544.21|
|Betty|5630.26|2008|         4815.7225|
|Betty|6222.57|2009|          5097.092|
|Betty|6623.97|2010| 5351.571666666667|
|  Dan|4262.75|2005|           4262.75|
|  Dan|4829.28|2006| 4546.014999999999|
|  Dan|5204.81|2007| 4765.613333333334|
|  Dan|6244.11|2008|         5135.2375|
|  Dan|7642.18|2009|          5636.626|


In [288]:
windowSpec = Window.partitionBy("name").orderBy("year").rangeBetween(Window.unboundedPreceding,0)

salaryHistory.withColumn("moving_sum", f.sum(f.col("salary")).over(windowSpec)).show(30)

+-----+-------+----+------------------+
| name| salary|year|        moving_sum|
+-----+-------+----+------------------+
|Chris|3601.42|2005|           3601.42|
|Chris|4015.66|2006|           7617.08|
|Chris|4304.73|2007|          11921.81|
|Chris|4650.33|2008|          16572.14|
|Chris|4932.86|2009|           21505.0|
|Chris|5869.98|2010|          27374.98|
| Greg|4226.89|2005|           4226.89|
| Greg|4935.72|2007|           9162.61|
| Greg|5274.04|2008|14436.650000000001|
| Greg|6227.36|2009|20664.010000000002|
| Greg|6033.33|2010|26697.340000000004|
|Betty|4138.01|2005|           4138.01|
|Betty|4376.94|2006|           8514.95|
|Betty|5117.68|2007|13632.630000000001|
|Betty|5630.26|2008|          19262.89|
|Betty|6222.57|2009|          25485.46|
|Betty|6623.97|2010|          32109.43|
|  Dan|4262.75|2005|           4262.75|
|  Dan|4829.28|2006| 9092.029999999999|
|  Dan|5204.81|2007|          14296.84|
|  Dan|6244.11|2008|          20540.95|
|  Dan|7642.18|2009|          28183.13|


> **TODO**: Wyświetl DF z kolumną zawierającą średnią wyliczaną z bieżącej, poprzedniej i kolejnej pensji dla poszczególnych osób (salaryHistory)

In [289]:
windowSpec = Window.partitionBy("name").orderBy("year")\
.rangeBetween(-1,1)

salaryHistory\
.withColumn("moving_avg", f.avg(f.col("salary")).over(windowSpec))\
.show(30)

+-----+-------+----+------------------+
| name| salary|year|        moving_avg|
+-----+-------+----+------------------+
|Chris|3601.42|2005|           3808.54|
|Chris|4015.66|2006|3973.9366666666665|
|Chris|4304.73|2007| 4323.573333333333|
|Chris|4650.33|2008| 4629.306666666666|
|Chris|4932.86|2009| 5151.056666666666|
|Chris|5869.98|2010|           5401.42|
| Greg|4226.89|2005|           4226.89|
| Greg|4935.72|2007|           5104.88|
| Greg|5274.04|2008|           5479.04|
| Greg|6227.36|2009|           5844.91|
| Greg|6033.33|2010| 6130.344999999999|
|Betty|4138.01|2005|          4257.475|
|Betty|4376.94|2006|           4544.21|
|Betty|5117.68|2007| 5041.626666666666|
|Betty|5630.26|2008| 5656.836666666667|
|Betty|6222.57|2009| 6158.933333333333|
|Betty|6623.97|2010|           6423.27|
|  Dan|4262.75|2005| 4546.014999999999|
|  Dan|4829.28|2006| 4765.613333333334|
|  Dan|5204.81|2007| 5426.066666666667|
|  Dan|6244.11|2008|            6363.7|
|  Dan|7642.18|2009|           7930.88|


##### rowsBetween(start, end)

Dodaje wewnątrz każdego okna zakres (offset) na którym zastosowana zostanie wskazana funkcja (remisy w sortowaniu traktuje jak funkcja row_number)

In [290]:
windowSpec = Window.partitionBy("name").orderBy("year").rowsBetween(Window.unboundedPreceding,0)

salaryHistory.withColumn("moving_avg", f.avg(f.col("salary")).over(windowSpec)).show(30)

+-----+-------+----+------------------+
| name| salary|year|        moving_avg|
+-----+-------+----+------------------+
|Chris|3601.42|2005|           3601.42|
|Chris|4015.66|2006|           3808.54|
|Chris|4304.73|2007|3973.9366666666665|
|Chris|4650.33|2008|          4143.035|
|Chris|4932.86|2009|            4301.0|
|Chris|5869.98|2010| 4562.496666666667|
| Greg|4226.89|2005|           4226.89|
| Greg|4935.72|2007|          4581.305|
| Greg|5274.04|2008| 4812.216666666667|
| Greg|6227.36|2009|5166.0025000000005|
| Greg|6033.33|2010| 5339.468000000001|
|Betty|4138.01|2005|           4138.01|
|Betty|4376.94|2006|          4257.475|
|Betty|5117.68|2007|           4544.21|
|Betty|5630.26|2008|         4815.7225|
|Betty|6222.57|2009|          5097.092|
|Betty|6623.97|2010| 5351.571666666667|
|  Dan|4262.75|2005|           4262.75|
|  Dan|4829.28|2006| 4546.014999999999|
|  Dan|5204.81|2007| 4765.613333333334|
|  Dan|6244.11|2008|         5135.2375|
|  Dan|7642.18|2009|          5636.626|


> **TODO**: Wyświetl DF z kolumną zawierającą średnią wyliczaną z dwóch poprzednich pensji dla poszczególnych osób (salaryHistory)

In [292]:
windowSpec = Window.partitionBy("name").orderBy("year")\
.rowsBetween(-2,-1)

salaryHistory\
.withColumn("moving_avg", f.avg(f.col("salary")).over(windowSpec))\
.show(30)

+-----+-------+----+------------------+
| name| salary|year|        moving_avg|
+-----+-------+----+------------------+
|Chris|3601.42|2005|              null|
|Chris|4015.66|2006|           3601.42|
|Chris|4304.73|2007|           3808.54|
|Chris|4650.33|2008|          4160.195|
|Chris|4932.86|2009|           4477.53|
|Chris|5869.98|2010| 4791.594999999999|
| Greg|4226.89|2005|              null|
| Greg|4935.72|2007|           4226.89|
| Greg|5274.04|2008|          4581.305|
| Greg|6227.36|2009|           5104.88|
| Greg|6033.33|2010|            5750.7|
|Betty|4138.01|2005|              null|
|Betty|4376.94|2006|           4138.01|
|Betty|5117.68|2007|          4257.475|
|Betty|5630.26|2008|4747.3099999999995|
|Betty|6222.57|2009|           5373.97|
|Betty|6623.97|2010|          5926.415|
|  Dan|4262.75|2005|              null|
|  Dan|4829.28|2006|           4262.75|
|  Dan|5204.81|2007| 4546.014999999999|
|  Dan|6244.11|2008|          5017.045|
|  Dan|7642.18|2009|           5724.46|


### SQL

Spark SQL wspiera standard SQL 2003

Aby użyć DF w zapytaniu SQL musimy go najpierw zarejestrować.

In [293]:
salaryHistory.createOrReplaceTempView("salaryHistory")
# salaryHistory.registerTempTable("salaryHistorySQL") - Spark 1.x

Proste zapytanie

In [294]:
spark.sql("select * from salaryHistory limit 2").show()

+-----+-------+----+
| name| salary|year|
+-----+-------+----+
|Alice|4404.23|2005|
|Alice| 3000.0|2005|
+-----+-------+----+



Zapytanie zwraca nowy DF

In [295]:
type(spark.sql("select * from salaryHistory limit 2"))

pyspark.sql.dataframe.DataFrame

Zapytanie z wykorzystaniem GROUP BY

In [296]:
spark.sql("select name, avg(salary) avg_sal from salaryHistory group by name").show()

+-----+-----------------+
| name|          avg_sal|
+-----+-----------------+
|Chris|4562.496666666667|
| Greg|5339.468000000001|
|Betty|5351.571666666667|
|  Dan|6348.246666666667|
|Alice|4949.138571428572|
+-----+-----------------+



Zapytanie z wykorzystaniem funkcji okiennej

In [297]:
spark.sql("select name, salary, avg(salary) over (partition by name) avg_sal from salaryHistory").show()

+-----+-------+-----------------+
| name| salary|          avg_sal|
+-----+-------+-----------------+
|Chris|3601.42|4562.496666666667|
|Chris|4015.66|4562.496666666667|
|Chris|4304.73|4562.496666666667|
|Chris|4650.33|4562.496666666667|
|Chris|4932.86|4562.496666666667|
|Chris|5869.98|4562.496666666667|
| Greg|4226.89|5339.468000000001|
| Greg|4935.72|5339.468000000001|
| Greg|5274.04|5339.468000000001|
| Greg|6227.36|5339.468000000001|
| Greg|6033.33|5339.468000000001|
|Betty|4138.01|5351.571666666667|
|Betty|4376.94|5351.571666666667|
|Betty|5117.68|5351.571666666667|
|Betty|5630.26|5351.571666666667|
|Betty|6222.57|5351.571666666667|
|Betty|6623.97|5351.571666666667|
|  Dan|4262.75|6348.246666666667|
|  Dan|4829.28|6348.246666666667|
|  Dan|5204.81|6348.246666666667|
+-----+-------+-----------------+
only showing top 20 rows



Usunięcie zbioru z katalogu

In [298]:
spark.catalog.dropTempView("salaryHistory")

> **TODO**: Wyświetl DF z kolumną zawierającą średnią wyliczaną z dwóch poprzednich pensji dla poszczególnych osób - zadanie analogiczne do zadania przy rowsBetween - zarejestruj DF, oraz napisz odpowiednie zapytanie (salaryHistory)

In [302]:
salaryHistory.createOrReplaceTempView("salaryHistory")

In [306]:
q = (
"select name, salary, year, avg(salary) over"
"(partition by name order by year rows between "
"2 preceding 1 preceding) moving_avg from salaryHistory"
)

spark.sql(q).show(20)

ParseException: "\nmissing 'AND' at '1'(line 1, pos 101)\n\n== SQL ==\nselect name, salary, year, avg(salary) over(partition by name order by year rows between 2 preceding 1 preceding) moving_avg from salaryHistory\n-----------------------------------------------------------------------------------------------------^^^\n"

### UDF - User Defined Function

**NIEWYDAJNE**

Używaj tylko w ostateczności

In [307]:
from pyspark.sql.types import IntegerType, StringType, FloatType

Definicja funkcji pythonowej

In [308]:
def power3(value):
    return(value**3)

In [309]:
power3(3)

27

Rejestracja jako UDF

In [310]:
udfPower3 = f.udf(power3, FloatType())

In [311]:
salaryHistory.select("salary", udfPower3(f.col("salary")).alias("power3")).show()

+-------+-------------+
| salary|       power3|
+-------+-------------+
|4404.23| 8.5429912E10|
| 3000.0|2.70000005E10|
|4780.34|1.09238657E11|
|4881.72|1.16337197E11|
|5280.86|1.47269894E11|
|5976.68| 2.1349122E11|
|6320.14|2.52452749E11|
|4138.01| 7.0855672E10|
|4376.94| 8.3851682E10|
|5117.68|1.34035358E11|
|5630.26|1.78478268E11|
|6222.57|2.40940253E11|
|6623.97|2.90639774E11|
|3601.42|  4.671123E10|
|4015.66| 6.4754627E10|
|4304.73| 7.9769666E10|
|4650.33|1.00566032E11|
|4932.86|1.20031814E11|
|5869.98|2.02259939E11|
|4262.75| 7.7458588E10|
+-------+-------------+
only showing top 20 rows



Rejestracja jako UDF - pozwala na stosowanie funkcji w zapytaniach SQL

In [312]:
spark.udf.register("power3", power3, FloatType())

In [313]:
spark.sql("select salary, power3(salary) power3 from salaryHistory").show()

+-------+-------------+
| salary|       power3|
+-------+-------------+
|4404.23| 8.5429912E10|
| 3000.0|2.70000005E10|
|4780.34|1.09238657E11|
|4881.72|1.16337197E11|
|5280.86|1.47269894E11|
|5976.68| 2.1349122E11|
|6320.14|2.52452749E11|
|4138.01| 7.0855672E10|
|4376.94| 8.3851682E10|
|5117.68|1.34035358E11|
|5630.26|1.78478268E11|
|6222.57|2.40940253E11|
|6623.97|2.90639774E11|
|3601.42|  4.671123E10|
|4015.66| 6.4754627E10|
|4304.73| 7.9769666E10|
|4650.33|1.00566032E11|
|4932.86|1.20031814E11|
|5869.98|2.02259939E11|
|4262.75| 7.7458588E10|
+-------+-------------+
only showing top 20 rows



In [314]:
def divide(x,y):
    return x / y

In [315]:
divide(10,2)

5.0

In [316]:
udfDivide = f.udf(divide, FloatType())

In [317]:
salaryHistory.select(udfDivide(f.col("salary"), f.col("year")).alias("nonsens")).show()

+---------+
|  nonsens|
+---------+
|2.1966233|
|1.4962593|
|2.3830209|
|2.4323468|
|2.6299105|
|2.9749527|
|3.1443481|
|2.0638454|
|2.1819243|
|2.5499153|
|2.8039143|
| 3.097347|
|3.2955074|
|1.7962195|
|2.0018246|
| 2.144858|
|2.3159013|
|2.4553807|
| 2.920388|
|2.1260598|
+---------+
only showing top 20 rows



In [318]:
spark.udf.register("divide", divide, FloatType())

In [319]:
spark.sql("select divide(salary, year) nonsens from salaryHistory").show()

+---------+
|  nonsens|
+---------+
|2.1966233|
|1.4962593|
|2.3830209|
|2.4323468|
|2.6299105|
|2.9749527|
|3.1443481|
|2.0638454|
|2.1819243|
|2.5499153|
|2.8039143|
| 3.097347|
|3.2955074|
|1.7962195|
|2.0018246|
| 2.144858|
|2.3159013|
|2.4553807|
| 2.920388|
|2.1260598|
+---------+
only showing top 20 rows



### Hive

Spark SQL wspiera HiveQL. <br>
Spark SQL wspiera rownież wczytawanie/zapisywanie danych bezpośrednio z/do Apache Hive.<br>

Wiecej informacji na temat integracji z Hive:<br>
https://spark.apache.org/docs/latest/sql-data-sources-hive-tables.html